In [1]:
import ee

In [3]:
## authenticates the session with EE
ee.Authenticate()

# creates the connection with EE. 
ee.Initialize()

In [4]:
# Define ROI for soil
roi_soil = ee.Geometry.Rectangle([163.078070, -77.625204, 163.07800, -77.625340])


In [5]:
# Pull the image for the soil endmember
soil1 = ee.Image('LANDSAT/LC08/C02/T2_TOA/LC08_055116_20231205').select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8'])


In [6]:
def get_brightness(image):
    """
    Compute the brightness of an image by summing the visible bands (B2, B3, B4, B5, B6, B7, B8).
    """
    brightness = image.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8']).reduce(ee.Reducer.sum())
    return brightness

In [7]:
def get_average_of_brightest(image, roi, n=10):
    """
    For the top n brightest pixels, calculate the mean value for each band.
    """
    # Select the top n brightest pixels
    brightest_points = select_brightest_pixels(image, roi, n)

    # Reduce columns (mean for each band) from the selected brightest points
    means = brightest_points.reduceColumns(
        reducer=ee.Reducer.mean(),
        selectors=['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8']
    ).get('mean')

    return ee.Dictionary(means)

In [8]:
def select_brightest_pixels(image, roi, n=10):
    """
    Select the n brightest pixels from the image based on brightness.
    """
    # Compute brightness for each pixel
    brightness = get_brightness(image)

    # Sample the image to get all pixels in the region of interest (ROI)
    sampled_points = brightness.sample(region=roi, scale=30, numPixels=1000)

    # Sort the points by brightness in descending order
    sorted_points = sampled_points.sort('sum', False)

    # Take the top n brightest points
    top_n_brightest_points = sorted_points.limit(n)

    return top_n_brightest_points


In [9]:
# Calculate the mean for soil endmember
soil_mean = soil1.reduceRegion(ee.Reducer.mean(), roi_soil, 30).values()


In [22]:
def spectral_unmixing(image, roi, n=10):
    """
    Perform spectral unmixing using the average values of the top n brightest pixels as endmembers.
    """
    # Get the average endmember values from the brightest pixels for each image
    average_endmembers = get_average_of_brightest(image, roi, n)

    # Pull the average brightest values for each band and put under soil_mean
    soil_mean_values = soil_mean 

    # Define endmembers: ice_mean and soil_mean
    endmembers = [average_endmembers, soil_mean_values]

    # Perform unmixing using the average endmembers
    unmixed_image = image.unmix(endmembers)

    return unmixed_image.set('system:time_start', image.get('system:time_start'))

In [21]:
# Define a mosaic-by-date function
def mosaic_by_date(imcol):
    """
    Create a mosaic for each unique date in the image collection.
    """
    imlist = imcol.toList(imcol.size())
    def get_date(image):
        return ee.Image(image).date().format("YYYY-MM-dd")

    unique_dates = imlist.map(lambda im: get_date(im)).distinct()

    def create_mosaic(date_str):
        date = ee.Date(date_str)
        mosaic = imcol.filterDate(date, date.advance(1, 'day')).mosaic()
        return mosaic.set({
            'system:time_start': date.millis(),
            'system:id': date.format('YYYY-MM-dd')
        })

    mosaic_imlist = unique_dates.map(create_mosaic)
    return ee.ImageCollection(mosaic_imlist)

In [23]:
# Define the region of interest (ROI) for the area to process
roi = ee.Geometry.Rectangle([163.0, -77.7, 163.5, -77.5])  # Define your ROI here



In [24]:
# Define start and end date for the image collection filter
start_date = "2016-03-06"
end_date = "2025-01-01"



In [25]:
# Example usage with an image collection (e.g., LANDSAT/LC08)
s2 = ee.ImageCollection('LANDSAT/LC08/C02/T2_TOA')\
    .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8'])\
    .filterDate(start_date, end_date)\
    .filterBounds(roi)\
    .sort('DATE_ACQUIRED')

# Create a mosaic by date
s3 = mosaic_by_date(s2)

In [26]:
# Clip all the images in the s3 collection down to the ROI
def clip_image(image):
    return image.clip(roi)


In [27]:
#apply clip to image collection
l8_clipped = s3.map(clip_image)

In [28]:
# Process each image in the collection
def process_images(image):
    # Apply spectral unmixing on the image
    unmixed_image = spectral_unmixing(image, roi)

    # Optionally reduce across bands (if spectral unmixing adds multiple bands)
    reduced_image = unmixed_image.reduce(ee.Reducer.mean())

    # Store the date as metadata
    date = image.date().format('YYYY-MM-dd')
    unmixed_image = reduced_image.set('acquired_date', date)

    return unmixed_image

In [31]:
# Apply the process_images function to the filtered image collection l8_clipped
processed_images = l8_clipped.map(process_images)

print(processed_images.first())

ee.Image({
  "functionInvocationValue": {
    "functionName": "Collection.first",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.map",
          "arguments": {
            "baseAlgorithm": {
              "functionDefinitionValue": {
                "argumentNames": [
                  "_MAPPING_VAR_0_0"
                ],
                "body": {
                  "functionInvocationValue": {
                    "functionName": "Element.set",
                    "arguments": {
                      "key": {
                        "constantValue": "acquired_date"
                      },
                      "object": {
                        "functionInvocationValue": {
                          "functionName": "Image.reduce",
                          "arguments": {
                            "image": {
                              "functionInvocationValue": {
                                "functionName": "E

In [30]:
# Print processed image information
first_image = processed_images.first()
print(first_image.getInfo())  # Print the first image's metadata

EEException: Collection.first: Error in map(ID=0):
Collection.reduceColumns: Need 1 inputs for Reducer.mean, got 7.